# Make the hourly weather and traffic db
I plan to do the following here
- pull in all of the hourly weather from 2015 to 2020 from openweather
- pull in all of the traffic from chicago data portal
- merge the two with a date as primary key
- also include cols for year month day

I will need to go back and do the following:
- add a shared key between traffic and 

In [ ]:
#!/usr/bin/env python

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from modules.myfuncs import *
from sodapy import Socrata
import numpy as np
from scipy import stats



In [ ]:
conn = create_connection('database/rlc.db')  # function from myfuncs file
c = conn.cursor()

In [ ]:
sql_fetch_tables(c, conn)

In [ ]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:

url = "data.cityofchicago.org"
client = Socrata(url, None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofchicago.org,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

In [ ]:
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# data website https://data.cityofchicago.org/resource/85ca-t3if.json

traffic_data = client.get("emtn-qqdi", 
                     where="TIME > \'2015-01-01T00:00:00.000\'",
                     limit=10000000,
                    )

# Convert to pandas DataFrame
traffic_data = pd.DataFrame.from_records(traffic_data)


In [6]:
traffic_data.head()

,time,region_id,bus_count,number_of_reads,speed
0,2015-01-01T00:01:17.000,23,37,332,28.64
1,2015-01-01T00:01:17.000,24,21,411,27.27
2,2015-01-01T00:01:17.000,6,17,303,25.23
3,2015-01-01T00:01:17.000,14,16,203,25.09
4,2015-01-01T00:01:17.000,18,24,432,27.89


In [25]:
traffic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4168199 entries, 0 to 4168198
Data columns (total 10 columns):
time               datetime64[ns]
region_id          object
bus_count          object
number_of_reads    object
speed              float64
year               int64
month              int64
day_of_week        int64
day                int64
hour               int64
dtypes: datetime64[ns](1), float64(1), int64(5), object(3)
memory usage: 318.0+ MB


In [26]:
# ex: df['Date']= pd.to_datetime(df['Date'])
traffic_data['time'] = pd.to_datetime(traffic_data['time'])

traffic_data['year'] = traffic_data['time'].apply(lambda x: x.year)
traffic_data['month'] = traffic_data['time'].apply(lambda x: x.month)
traffic_data['day_of_week'] = traffic_data['time'].dt.dayofweek

# need to convert this to the dataset convention 1 for Sunday, 7 for Saturday
traffic_data['day_of_week'] = traffic_data['day_of_week'].apply(lambda x: (x + 2) % 7)

traffic_data['day'] = traffic_data['time'].apply(lambda x: x.day)
traffic_data['hour'] = traffic_data['time'].apply(lambda x: x.hour)
traffic_data['speed'] = traffic_data['speed'].apply(float)

In [27]:
traffic_data.head()

,time,region_id,bus_count,number_of_reads,speed,year,month,day_of_week,day,hour
0,2015-01-01 00:01:17,23,37,332,28.64,2015,1,5,1,0
1,2015-01-01 00:01:17,24,21,411,27.27,2015,1,5,1,0
2,2015-01-01 00:01:17,6,17,303,25.23,2015,1,5,1,0
3,2015-01-01 00:01:17,14,16,203,25.09,2015,1,5,1,0
4,2015-01-01 00:01:17,18,24,432,27.89,2015,1,5,1,0


In [28]:
# now make it only have one entry per hour
agg_dict = {'speed':np.mean}
traffic_grouped = traffic_data.groupby(['year', 'month', 'day', 'hour', 'region_id', 'day_of_week']).agg(agg_dict).reset_index()


In [29]:
traffic_grouped.head(20)  # YES!!! That's what I want

,year,month,day,hour,region_id,day_of_week,speed
0,2015,1,1,0,1,5,27.455000
1,2015,1,1,0,10,5,25.796667
2,2015,1,1,0,11,5,25.816667
3,2015,1,1,0,12,5,18.636667
4,2015,1,1,0,13,5,20.681667
5,2015,1,1,0,14,5,26.408333
6,2015,1,1,0,15,5,26.035000
7,2015,1,1,0,16,5,39.313333
8,2015,1,1,0,17,5,21.648333
9,2015,1,1,0,18,5,26.931667


In [30]:
traffic_grouped[traffic_grouped['year']==2018]['month'].max()  # this data only goes up to May 2018
# we have an overlap in the dbs from March through April of 2018.  
# WIll result in double entries, and we will have to grouby mean speed again to fix it.
# It makes sense to just


5

In [31]:
#  results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# data website https://data.cityofchicago.org/resource/85ca-t3if.json

# 15 MINUTES

new_traffic_data = client.get("kf7e-cur8", #2018 to present
                     where="TIME < \'2021-01-01T00:00:00.000\'",
                     limit=10000000,
                    )

# Convert to pandas DataFrame
new_traffic_data = pd.DataFrame.from_records(new_traffic_data)

In [32]:
new_traffic_data.head()

,time,region_id,speed,region,bus_count,num_reads,hour,day_of_week,month,description,record_id,west,east,south,north,nw_location,se_location
0,2018-03-31T14:40:28.000,2,27.95,Far North West,25,493,14,7,3,North of Montrose. East River to Cicero,02-201803311940,-87.84621,-87.747456,41.960669,42.0190998,"{'type': 'Point', 'coordinates': [-87.84621, 4...","{'type': 'Point', 'coordinates': [-87.747456, ..."
1,2018-03-31T14:40:28.000,3,19.09,North Park-Albany-Linconl Sq,48,834,14,7,3,Montrose to Devon. Cicero to Ravenswood,03-201803311940,-87.747456,-87.67459,41.960669,41.997946,"{'type': 'Point', 'coordinates': [-87.747456, ...","{'type': 'Point', 'coordinates': [-87.67459, 4..."
2,2018-03-31T14:40:28.000,4,18.41,Edge Water-Uptown,36,585,14,7,3,Montrose to Devon. Ravenswood to Lake Shore,04-201803311940,-87.67459,-87.646438,41.960669,41.997946,"{'type': 'Point', 'coordinates': [-87.67459, 4...","{'type': 'Point', 'coordinates': [-87.646438, ..."
3,2018-03-31T14:40:28.000,6,20.45,Irving Park-Avondale-North Ctr,48,811,14,7,3,Diversey to Montrose. Cicero to Ravenswood,06-201803311940,-87.747456,-87.67459,41.931841,41.960669,"{'type': 'Point', 'coordinates': [-87.747456, ...","{'type': 'Point', 'coordinates': [-87.67459, 4..."
4,2018-03-31T14:40:28.000,7,18.41,Hermosa-Logan Square,47,817,14,7,3,North Ave to Diversy. Cicero to Ravenswood,07-201803311940,-87.747456,-87.67459,41.909269,41.931841,"{'type': 'Point', 'coordinates': [-87.747456, ...","{'type': 'Point', 'coordinates': [-87.67459, 4..."


In [33]:
try:
    new_traffic_data.drop(columns=['region', 'bus_count', 'num_reads', 'record_id', 'nw_location', 'se_location'], inplace=True)
except:
    pass
new_traffic_data.head()

,time,region_id,speed,hour,day_of_week,month,description,west,east,south,north
0,2018-03-31T14:40:28.000,2,27.95,14,7,3,North of Montrose. East River to Cicero,-87.84621,-87.747456,41.960669,42.0190998
1,2018-03-31T14:40:28.000,3,19.09,14,7,3,Montrose to Devon. Cicero to Ravenswood,-87.747456,-87.67459,41.960669,41.997946
2,2018-03-31T14:40:28.000,4,18.41,14,7,3,Montrose to Devon. Ravenswood to Lake Shore,-87.67459,-87.646438,41.960669,41.997946
3,2018-03-31T14:40:28.000,6,20.45,14,7,3,Diversey to Montrose. Cicero to Ravenswood,-87.747456,-87.67459,41.931841,41.960669
4,2018-03-31T14:40:28.000,7,18.41,14,7,3,North Ave to Diversy. Cicero to Ravenswood,-87.747456,-87.67459,41.909269,41.931841


In [ ]:
# ex: df['Date']= pd.to_datetime(df['Date'])
new_traffic_data['time'] = pd.to_datetime(new_traffic_data['time'])
new_traffic_data['year'] = new_traffic_data['time'].apply(lambda x: int(x.year))
new_traffic_data['day'] = new_traffic_data['time'].apply(lambda x: int(x.day))
new_traffic_data['hour'] = new_traffic_data['hour'].apply(int)

new_traffic_data['speed'] = new_traffic_data['speed'].apply(float)


In [46]:
# now make it only have one entry per hour
# This code takes about 10min to run
new_traffic_grouped = new_traffic_data.groupby(['year', 'month', 'day', 'hour', 'region_id', 'day_of_week']).agg({'speed':np.mean}).reset_index()


In [ ]:
new_traffic_grouped.head()
new_traffic_grouped.info()

In [ ]:
new_traffic_grouped[new_traffic_grouped['year']==2018]['month'].min()  # this data only goes back to March 2018
# looks like we have a gap of three months.  Should fill it in with dummy data.

# now merge the two datasets (2015-2018 and 2018-2020)

In [42]:
all_traffic_grouped = pd.concat([traffic_grouped, new_traffic_grouped], ignore_index=False)

In [43]:
all_traffic_grouped.head()

,year,month,day,hour,region_id,day_of_week,speed
0,2015,1,1,0,1,5,27.455000
1,2015,1,1,0,10,5,25.796667
2,2015,1,1,0,11,5,25.816667
3,2015,1,1,0,12,5,18.636667
4,2015,1,1,0,13,5,20.681667


In [44]:
print(len(all_traffic_grouped))
print(len(traffic_grouped) + len(new_traffic_grouped))  # concatenation successful
print(all_traffic_grouped['hour'].unique())

1368684
1368684
[0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 '0' '1'
 '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22' '23'
 '3' '4' '5' '6' '7' '8' '9']


In [ ]:
traffic_grouped.info()
new_traffic_grouped.info()

In [ ]:
all_traffic_grouped['day_of_week'].isna().sum()

In [ ]:
all_traffic_grouped[(all_traffic_grouped['year']==2018) & \
                    (all_traffic_grouped['month']==4) & \
                    (all_traffic_grouped['day']==5) & \
                    (all_traffic_grouped['hour']==5) & \
                    (all_traffic_grouped['region_id']=='1')
                   ]

# I have some doubles.  My problem is the WRONG DAY OF WEEK!!!!
# Thursday is the correct one here.


traffic_grouped[(traffic_grouped['year']==2018) & \
                (traffic_grouped['month']==4) & \
                (traffic_grouped['day']==5) & \
                (traffic_grouped['hour']==5) & \
                (traffic_grouped['region_id']=='1')
               ]

new_traffic_grouped[(new_traffic_grouped['year']==2018) & \
                    (new_traffic_grouped['month']==4) & \
                    (new_traffic_grouped['day']==5) & \
                    (new_traffic_grouped['hour']==5) & \
                    (new_traffic_grouped['region_id']=='1')
                   ]

# for new data, sunday = 1, so thursday = 5 which is the correct number.
# From datetime docs:
# dayofweek method Return the dayof the week. 
# It is assumed the week starts on Monday, which is denoted by 0 and ends on Sunday which is denoted by 6. 
# So let's go back and make it all the same.  I prefer to use the 0 Sunday (traffic_grouped).  
# Will Go back above and fix new_traffic by subtracting

In [ ]:
all_traffic_grouped = all_traffic_grouped \
                    .groupby(['year', 'month', 'day', 'hour', 'region_id', 'day_of_week']) \
                    .agg({'speed':np.mean}) \
                    .reset_index()

In [ ]:
print(24 * 29 * 365 * 5) # estimated total entries (24 hours, 29 regions, 365 days, 5 years)
print(len(all_traffic_grouped)) # difference is 
all_traffic_grouped.head() 


In [ ]:
print(new_traffic_grouped['year'].min(), new_traffic_grouped['year'].max())  # this data only goes back to March 2018
print(traffic_grouped['year'].min(), traffic_grouped['year'].max())
print(all_traffic_grouped['year'].min(), all_traffic_grouped['year'].max())


      